In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

In [2]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
from transformers import pipeline

In [9]:
roberta = pipeline(model = 'tired-racoon/tonika_sentim')

Device set to use cuda:0


In [12]:
ds_500 = pd.read_excel('dataset_comments_LABELED.xlsx')
ds_500

,UserSenderId,SubmitDate,MessageText,Class,Unnamed: 4,Unnamed: 5
0,14771,2023-05-16 19:27:21.773,"<p class=""st0"">В общем podman ...",N,NaN,NaN
1,8194,2023-05-16 16:56:35.380,"<p class=""st0"">А зачем ввод ИНН? Если его нет ...",N,NaN,NaN
2,18355,2023-05-16 15:08:17.097,сориентируйте по размерам пожалуйста,N,NaN,NaN
3,18551,2023-05-16 09:33:51.050,"толстовка хорошая, но начес лезет сильно. над...",N,NaN,NaN
4,14634,2023-05-15 14:21:37.207,"<p class=""st0"">Здорово! Нащупали идеальный фор...",G,NaN,NaN
...,...,...,...,...,...,...
503,8340,2018-12-12 09:10:29.027,"<p class=""st0"">Классная идея!!!)</p>",G,NaN,NaN
504,14771,2018-12-11 11:16:14.397,Бессмысленный товар. Я думал светится хотя бы ...,B,NaN,NaN
505,9190,2018-12-11 09:16:55.903,"<p class=""st0"">Коллеги, большое спасибо за поз...",G,NaN,NaN
506,8400,2018-12-11 08:45:17.930,"<p class=""st0"">Вчера прикупил. Очень вкусно.</p>",G,NaN,NaN


In [13]:
from bs4 import BeautifulSoup


ds_500['text'] = ds_500['MessageText'].fillna('').apply(
    lambda x: BeautifulSoup(x, "html.parser").get_text(strip=True)[:512]
)

# ds_500.iloc[0]['text']

C:\Users\ptarh\AppData\Local\Temp\ipykernel_27120\421802642.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  lambda x: BeautifulSoup(x, "html.parser").get_text(strip=True)[:512]


In [14]:
result = roberta(list(ds_500['text']))
result_top3 = roberta(list(ds_500['text']), top_k = 3)

In [15]:
result, result_top3

([{'label': 'LABEL_0', 'score': 0.9004085659980774},
  {'label': 'LABEL_1', 'score': 0.6956427097320557},
  {'label': 'LABEL_1', 'score': 0.9982398748397827},
  {'label': 'LABEL_2', 'score': 0.48613548278808594},
  {'label': 'LABEL_2', 'score': 0.8207626342773438},
  {'label': 'LABEL_1', 'score': 0.9982801675796509},
  {'label': 'LABEL_2', 'score': 0.7312572598457336},
  {'label': 'LABEL_1', 'score': 0.9256969094276428},
  {'label': 'LABEL_1', 'score': 0.9364262819290161},
  {'label': 'LABEL_2', 'score': 0.9707057476043701},
  {'label': 'LABEL_2', 'score': 0.9843963384628296},
  {'label': 'LABEL_2', 'score': 0.9515624046325684},
  {'label': 'LABEL_1', 'score': 0.9914318323135376},
  {'label': 'LABEL_2', 'score': 0.8484768271446228},
  {'label': 'LABEL_2', 'score': 0.8652252554893494},
  {'label': 'LABEL_1', 'score': 0.9785485863685608},
  {'label': 'LABEL_1', 'score': 0.9977598190307617},
  {'label': 'LABEL_1', 'score': 0.9637373685836792},
  {'label': 'LABEL_1', 'score': 0.52359235286

In [16]:
LABEL_MAPPING = {
    "LABEL_0": "B",
    "LABEL_1": "N",
    "LABEL_2": "G",
}

In [24]:
top_score = []
pred_class = []
for pr in result:
    pred_class.append(pr['label'])
    top_score.append(pr['score'])

ds_500['pred_class'] = pred_class
ds_500['top_score'] = top_score
ds_500['pred_class'] = ds_500['pred_class'].map(LABEL_MAPPING)
ds_500

,UserSenderId,SubmitDate,MessageText,Class,Unnamed: 4,Unnamed: 5,text,pred_class,top_score
0,14771,2023-05-16 19:27:21.773,"<p class=""st0"">В общем podman ...",N,NaN,NaN,В общем podman - это не наш метод. На нем рабо...,B,0.900409
1,8194,2023-05-16 16:56:35.380,"<p class=""st0"">А зачем ввод ИНН? Если его нет ...",N,NaN,NaN,"А зачем ввод ИНН? Если его нет у компании, то ...",N,0.695643
2,18355,2023-05-16 15:08:17.097,сориентируйте по размерам пожалуйста,N,NaN,NaN,сориентируйте по размерам пожалуйста,N,0.998240
3,18551,2023-05-16 09:33:51.050,"толстовка хорошая, но начес лезет сильно. над...",N,NaN,NaN,"толстовка хорошая, но начес лезет сильно. над...",G,0.486135
4,14634,2023-05-15 14:21:37.207,"<p class=""st0"">Здорово! Нащупали идеальный фор...",G,NaN,NaN,Здорово! Нащупали идеальный формат 😎👍👍Из предл...,G,0.820763
...,...,...,...,...,...,...,...,...,...
503,8340,2018-12-12 09:10:29.027,"<p class=""st0"">Классная идея!!!)</p>",G,NaN,NaN,Классная идея!!!),G,0.985095
504,14771,2018-12-11 11:16:14.397,Бессмысленный товар. Я думал светится хотя бы ...,B,NaN,NaN,Бессмысленный товар. Я думал светится хотя бы ...,B,0.982852
505,9190,2018-12-11 09:16:55.903,"<p class=""st0"">Коллеги, большое спасибо за поз...",G,NaN,NaN,"Коллеги, большое спасибо за поздравления :)",G,0.977989
506,8400,2018-12-11 08:45:17.930,"<p class=""st0"">Вчера прикупил. Очень вкусно.</p>",G,NaN,NaN,Вчера прикупил. Очень вкусно.,G,0.891444


In [25]:
ds_500.columns

Index(['UserSenderId', 'SubmitDate', 'MessageText', 'Class', 'Unnamed: 4',
       'Unnamed: 5', 'text', 'pred_class', 'top_score'],
      dtype='object')

In [26]:
ds_500.drop(['Unnamed: 4', 'Unnamed: 5'], axis = 1, inplace = True)
ds_500

,UserSenderId,SubmitDate,MessageText,Class,text,pred_class,top_score
0,14771,2023-05-16 19:27:21.773,"<p class=""st0"">В общем podman ...",N,В общем podman - это не наш метод. На нем рабо...,B,0.900409
1,8194,2023-05-16 16:56:35.380,"<p class=""st0"">А зачем ввод ИНН? Если его нет ...",N,"А зачем ввод ИНН? Если его нет у компании, то ...",N,0.695643
2,18355,2023-05-16 15:08:17.097,сориентируйте по размерам пожалуйста,N,сориентируйте по размерам пожалуйста,N,0.998240
3,18551,2023-05-16 09:33:51.050,"толстовка хорошая, но начес лезет сильно. над...",N,"толстовка хорошая, но начес лезет сильно. над...",G,0.486135
4,14634,2023-05-15 14:21:37.207,"<p class=""st0"">Здорово! Нащупали идеальный фор...",G,Здорово! Нащупали идеальный формат 😎👍👍Из предл...,G,0.820763
...,...,...,...,...,...,...,...
503,8340,2018-12-12 09:10:29.027,"<p class=""st0"">Классная идея!!!)</p>",G,Классная идея!!!),G,0.985095
504,14771,2018-12-11 11:16:14.397,Бессмысленный товар. Я думал светится хотя бы ...,B,Бессмысленный товар. Я думал светится хотя бы ...,B,0.982852
505,9190,2018-12-11 09:16:55.903,"<p class=""st0"">Коллеги, большое спасибо за поз...",G,"Коллеги, большое спасибо за поздравления :)",G,0.977989
506,8400,2018-12-11 08:45:17.930,"<p class=""st0"">Вчера прикупил. Очень вкусно.</p>",G,Вчера прикупил. Очень вкусно.,G,0.891444


In [29]:
label_column = {
    "LABEL_0": "bad",
    "LABEL_1": "neutral",
    "LABEL_2": "good",
}
bads = []
goods = []
neuts = []
for lst in result_top3:
    for elem in lst:
        col = label_column.get(elem['label'])
        if col == "bad":
            bads.append(elem['score'])
        elif col == "good":
            goods.append(elem['score'])
        else:
            neuts.append(elem['score'])

In [30]:
ds_500['good'] = goods
ds_500['bad'] = bads
ds_500['neutral'] = neuts
ds_500

,UserSenderId,SubmitDate,MessageText,Class,text,pred_class,top_score,good,bad,neutral
0,14771,2023-05-16 19:27:21.773,"<p class=""st0"">В общем podman ...",N,В общем podman - это не наш метод. На нем рабо...,B,0.900409,0.056626,0.900409,0.042965
1,8194,2023-05-16 16:56:35.380,"<p class=""st0"">А зачем ввод ИНН? Если его нет ...",N,"А зачем ввод ИНН? Если его нет у компании, то ...",N,0.695643,0.004826,0.299532,0.695643
2,18355,2023-05-16 15:08:17.097,сориентируйте по размерам пожалуйста,N,сориентируйте по размерам пожалуйста,N,0.998240,0.001041,0.000719,0.998240
3,18551,2023-05-16 09:33:51.050,"толстовка хорошая, но начес лезет сильно. над...",N,"толстовка хорошая, но начес лезет сильно. над...",G,0.486135,0.486135,0.307707,0.206158
4,14634,2023-05-15 14:21:37.207,"<p class=""st0"">Здорово! Нащупали идеальный фор...",G,Здорово! Нащупали идеальный формат 😎👍👍Из предл...,G,0.820763,0.820763,0.003751,0.175486
...,...,...,...,...,...,...,...,...,...,...
503,8340,2018-12-12 09:10:29.027,"<p class=""st0"">Классная идея!!!)</p>",G,Классная идея!!!),G,0.985095,0.985095,0.001508,0.013397
504,14771,2018-12-11 11:16:14.397,Бессмысленный товар. Я думал светится хотя бы ...,B,Бессмысленный товар. Я думал светится хотя бы ...,B,0.982852,0.006593,0.982852,0.010555
505,9190,2018-12-11 09:16:55.903,"<p class=""st0"">Коллеги, большое спасибо за поз...",G,"Коллеги, большое спасибо за поздравления :)",G,0.977989,0.977989,0.002208,0.019802
506,8400,2018-12-11 08:45:17.930,"<p class=""st0"">Вчера прикупил. Очень вкусно.</p>",G,Вчера прикупил. Очень вкусно.,G,0.891444,0.891444,0.006857,0.101699


## Эксперимент 1

Используем все данные, не только с низким скором

In [33]:
X = ds_500[['text', 'pred_class', 'top_score', 'good', 'bad', 'neutral']]
y = ds_500['Class']

In [35]:
from catboost import CatBoostClassifier
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

In [86]:
cat = CatBoostClassifier(random_state = 42, n_estimators = 170, learning_rate = 0.5)

cat.fit(X_train, y_train, text_features = ['text'], cat_features = ['pred_class'])
pred = cat.predict(X_test)

0:	learn: 0.7600444	total: 44.3ms	remaining: 7.48s
1:	learn: 0.6507585	total: 98.3ms	remaining: 8.26s
2:	learn: 0.6058336	total: 146ms	remaining: 8.1s
3:	learn: 0.5812580	total: 193ms	remaining: 8.01s
4:	learn: 0.5233888	total: 241ms	remaining: 7.96s
5:	learn: 0.5041494	total: 291ms	remaining: 7.94s
6:	learn: 0.4778264	total: 327ms	remaining: 7.62s
7:	learn: 0.4601294	total: 375ms	remaining: 7.6s
8:	learn: 0.4519016	total: 423ms	remaining: 7.56s
9:	learn: 0.4429841	total: 479ms	remaining: 7.67s
10:	learn: 0.4291333	total: 545ms	remaining: 7.87s
11:	learn: 0.4086355	total: 593ms	remaining: 7.81s
12:	learn: 0.3994914	total: 641ms	remaining: 7.75s
13:	learn: 0.3921223	total: 689ms	remaining: 7.68s
14:	learn: 0.3823572	total: 737ms	remaining: 7.61s
15:	learn: 0.3762797	total: 785ms	remaining: 7.56s
16:	learn: 0.3683656	total: 834ms	remaining: 7.51s
17:	learn: 0.3609806	total: 881ms	remaining: 7.44s
18:	learn: 0.3431952	total: 929ms	remaining: 7.38s
19:	learn: 0.3375737	total: 978ms	remaini

In [88]:
recall_score(pred, y_test, average = 'micro')

0.803921568627451

In [90]:
cat.save_model('model_all_score804')

## Эксперимент 2

Обучаемся только на низких скорах

In [163]:
trhd = 0.9
dataset_low = ds_500[ds_500['top_score'] < trhd]
X = dataset_low[['text', 'pred_class', 'top_score', 'good', 'bad', 'neutral']]
y = dataset_low['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

In [164]:
# Посмотрим, как много у нас галлюцинаций - когда модель слишком уверена в неверном ответе

len(ds_500[(ds_500['top_score'] > trhd) & (ds_500['Class'] != ds_500['pred_class'])])

62

In [168]:
# А теперь - как много раз мы ошиблись из-за неуверенности

len(ds_500[(ds_500['top_score'] < trhd) & (ds_500['Class'] != ds_500['pred_class'])])

57

In [165]:
cat = CatBoostClassifier(random_state = 42, n_estimators = 280, learning_rate = 0.5)

cat.fit(X_train, y_train, text_features = ['text'], cat_features = ['pred_class'])
pred = cat.predict(X_test)

0:	learn: 0.8859579	total: 40.4ms	remaining: 11.3s
1:	learn: 0.7651997	total: 76.5ms	remaining: 10.6s
2:	learn: 0.7003814	total: 112ms	remaining: 10.4s
3:	learn: 0.6511318	total: 145ms	remaining: 10s
4:	learn: 0.5900488	total: 181ms	remaining: 9.97s
5:	learn: 0.5589226	total: 215ms	remaining: 9.8s
6:	learn: 0.5205960	total: 250ms	remaining: 9.74s
7:	learn: 0.5134838	total: 292ms	remaining: 9.94s
8:	learn: 0.5010362	total: 328ms	remaining: 9.86s
9:	learn: 0.4892995	total: 363ms	remaining: 9.81s
10:	learn: 0.4630413	total: 400ms	remaining: 9.77s
11:	learn: 0.4386709	total: 436ms	remaining: 9.73s
12:	learn: 0.4171334	total: 473ms	remaining: 9.72s
13:	learn: 0.3976285	total: 512ms	remaining: 9.73s
14:	learn: 0.3785409	total: 553ms	remaining: 9.77s
15:	learn: 0.3625262	total: 587ms	remaining: 9.69s
16:	learn: 0.3361046	total: 624ms	remaining: 9.65s
17:	learn: 0.3262282	total: 661ms	remaining: 9.62s
18:	learn: 0.3159979	total: 696ms	remaining: 9.56s
19:	learn: 0.3047865	total: 734ms	remainin

In [166]:
recall_score(pred, y_test, average = 'micro')

0.8709677419354839

In [156]:
cat.save_model('model_low_score871')

## Итоги

Мы создали модель, которая показывает recall выше 0.87 на тех данных, в которых неуверена большая модель. Есть "галлюцинации", но они очень сильно зависят от разметки и размера датасета, поскольку она в конкретной задаче достаточно субъективная. 